The second step is completing the 4-step IBC connection handshake between terra and osmosis - 1) init (on terra), 2) try (on osmosis), 3) ack (on terra), 4) confirm (on osmosis).

With an IBC connection in-place, we lay the foundation for terra/osmosis smart contract pairs to communicate.

In [1]:
#imports

In [ ]:
#setup lcd clients, rpc urls, wallets

These are the protobuf classes specific for setting up the IBC connection. One noteworthy protobuf class that will see continual usage now, and in later stages, is MsgUpdateClient, which basically refreshes the light client information (ie the counterpart chain tendermint state).

In [ ]:
#terra_proto imports

Load client_id's from previous step; if we don't find it, we'll use default client_id's built by the authors.

In [ ]:
#load context.yaml

IBC connection step 1 - MsgConnectionOpenInit

In [2]:
#fabricate message & dispatch via stargate

IBC connection step 2 - MsgConnectionOpenTry

In [ ]:
#update the client_id on osmosis (ie, refresh osmosis's knowledge of terra's tendermint state)

In [ ]:
#fetch client, connection, consensus proofs

In [ ]:
#fabricate message & dispatch via stargate

IBC connection step 3 - MsgConnectionOpenAck

In [ ]:
#update client on terra (ie, refresh terra's knowledge of osmosis's tendermint state) & fetch proofs

In [ ]:
#fabricate message & dispatch via stargate

IBC connection step 4 - MsgConnectionOpenConfirm

In [3]:
#update client again on osmosis & fetch proofs

In [ ]:
#fabricate message & dispatch via stargate

In [ ]:
#persist connection_id's on both chains for later stages